In [1]:
import numpy as np
from functools import partial
import sobol_seq

**Problem 1 Pak Kun**

In [2]:
def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    return np.array([f1,f2])

dim = 2
boundaries = np.array([(-10,10) for _ in range (dim)])
pop_size=250
max_gen=1000
F_init=0.9
CR_init=0.8
num_l=20
epsilon=1e-3
tau_d=0.4
s_max=100
print_gen=True
Hm = 50


# Objective Function

In [3]:
def root_objective_function(x:np.ndarray):
    res = 0
    F_array = objective_function(x)
    for f in F_array:
        res +=(f)**2
    return res

# def root_objective_function(x:np.ndarray):
#     F_array = objective_function(x)
#     denom = 0
#     for f in F_array:
#         denom += np.abs(f)
#     res = 1/(1+denom)
#     return -res

In [4]:
"""Characteristic Function"""
def chi_p(delta_j, rho):
    return 1 if delta_j <= rho else 0

"""Repulsion Function"""
def repulsion_function(x,
                       archive,
                       objective_func=root_objective_function,
                       beta=100,
                       rho=1e-8):
    f_x = objective_func(x)
    Rx = 0
    for x_star in archive:
        delta_j = np.linalg.norm(x-x_star)
        Rx += np.exp(-delta_j) * chi_p(delta_j, rho)
    Rx *= beta
    Rx += f_x
    return Rx

"""Fitness Function"""
def fitness_function(x,
                     archive,
                     objective_func=root_objective_function,
                     repulsion_func=repulsion_function):
    f_x = objective_func(x)
    if archive == []:
        return f_x
    else:
        return repulsion_func(x,archive)
    
# # Test the Functiuons
# arch = [np.array([1,2]),np.array([3,4]),np.array([-6.43,0.15])]
# x = np.array([-6.437160,0.155348])
# print(fitness_function(x,
#                        arch,
#                        objective_func=root_objective_function,
#                        repulsion_func=partial(repulsion_function,
#                                               objective_func = root_objective_function,
#                                               beta=1000,rho=0.01)))

In [5]:
def generate_points(dim: int,
                    npoint:int,
                    low=-10,
                    high=10,
                    sobol = True):
    """
    Generate points with option to use sobol sequence
    """
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    if sobol == True:
        # Generate Sobol sequence points
        sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)
        # Scale the Sobol points to fit within the specified boundaries
        scaled_points = []
        for i in range(dim):
            a, b = boundaries[i]
            scaled_dim = a + sobol_points[:, i] * (b - a)
            scaled_points.append(scaled_dim)
        # Transpose the scaled points to get points per dimension
        scaled_points = np.array(list(map(list, zip(*scaled_points))))
    
    else:
        scaled_points = np.zeros((npoint, dim))
        for i in range(dim):
            min_val, max_val = boundaries[i]
            scaled_points[:, i] = np.random.uniform(min_val, max_val, npoint)

    return scaled_points


## Spiral Optimization Algorithm

In [6]:
def generate_Rij(i,j,dim,theta):
    Rn_ij= np.eye(dim)
    Rn_ij[i-1,i-1] = np.cos(theta)
    Rn_ij[i-1,j-1] = -np.sin(theta)
    Rn_ij[j-1,i-1] = np.sin(theta)
    Rn_ij[j-1,j-1] = np.cos(theta)
    return Rn_ij

def generate_Rn(dim,theta):
    Rn = np.eye(dim)
    for i in range(0,dim):
        for j in range (0,i+1):
            product = np.eye(dim)
            product *= generate_Rij(dim-i-1,dim+1-j-1,dim,theta)
        Rn *= product
    return Rn

def minimize(set_of_points,objective_function):
    z = []
    z_min = 10**100
    for i in range (len(set_of_points)):
        z.append(objective_function(set_of_points.T)[i])
        if z[i]<z_min:
            z_min = z[i]
            idx_min = i
    x_min = set_of_points[idx_min]
    return z_min,idx_min,x_min

def update_point(set_of_points,Sn,dim,objective_function):
    (z_star,idx_star,x_star) = minimize(set_of_points,objective_function)
    new_set_of_points = np.copy(set_of_points)
    for i in range (len(new_set_of_points)):
        # perkalian matriks
        poin = np.dot(Sn,set_of_points[i].reshape(-1,1)) - np.dot((Sn-np.identity(dim)),x_star.reshape(-1,1))
        new_set_of_points[i] = poin.T
    return new_set_of_points

def iter_error(set_of_points,iter,npoint):
    err = 0
    for i in range (npoint):
        diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))
        if diff>err:
            err = diff
    return err

In [7]:
def SpiralOpt(objective_function, 
              set_of_points,
              r = 0.95,
              theta=np.pi/4, 
              iter_max=100, 
              error_max = 10**(-5),
              seed=0, 
              show_err=False, 
              show_objective_function=False):
    
    np.random.seed(seed)
    iter_points = {}
    iter = 0
    iter_points[iter] = set_of_points
    npoint = len(set_of_points)
    dim = len(set_of_points[0])
    Rn = generate_Rn(dim,theta)
    Sn = r*Rn
    while iter <= iter_max :
        iter_points[iter+1] = update_point(iter_points[iter],Sn,dim,objective_function)
        error = iter_error(iter_points,iter+1,npoint)
        if error < error_max:
            break
        iter += 1

    return_points = [iter_points[iter][:,i].mean() for i in range(dim)]
    if show_err == True:
        print(error)
    obj_fun = objective_function(iter_points[iter].T).mean()
    if show_objective_function ==True:
        print(obj_fun)
    return return_points, obj_fun


In [8]:
def update_archive(x: np.ndarray,
                   objective_function,
                   archive,
                   epsilon,
                   tau_d,
                   s_max):
    """Input"""
    # x : Individual
    # theta : accuracy level
    # tau_d : distance radius
    # s_max : maximum archive size
    # archive : archive
    # s : archive current size

    f_x = objective_function(x)
    s = len(archive) # archive current size
    if f_x < epsilon: # x is a root
        # print(f'f({x})= {f_x}')
        if s == 0: # archive is empty
            archive.append(x)
            s+=1
        else:
            """Find the closest solution x_prime ∈ archive to x in the decision space"""
            dist_min = np.linalg.norm(x-archive[0])
            idx_min = 0
            x_prime= archive[idx_min]
            for i in range(1,len(archive)): 
                dist = np.linalg.norm(x-archive[i])
                if dist < dist_min:
                    dist_min = dist
                    x_prime = archive[i]
                    idx_min = i
            f_x_prime = root_objective_function(x_prime)
            if dist_min < tau_d: # x and x_prime are too close
                if f_x < f_x_prime:
                    x_prime = x
                    archive[idx_min] = x_prime
            else:
                if s < s_max:
                    archive.append(x)
                    s += 1
                else:       # archive is full
                    if f_x<f_x_prime:
                        x_prime = x
                        archive[idx_min] = x_prime
    return archive


# # Test the function
# x = np.array([-6.437160,0.155348]) # Individual
# theta = 1e-4 # accuracy level
# tau_d = 1e-1 # distance radius
# s_max = 3 # maximum archive size
# archive = [np.array([0,0]),np.array([1,2]),np.array([-6.4,0])] # archive
# update_archive(x,root_objective_function,archive,theta,tau_d,s_max)

In [9]:
"""Calculate Euclidean distances and select t closest individuals"""
def subpopulating(individual, 
                  population, 
                  t,
                  return_index = False,
                  show_distances = False): 
    """Input"""
    # individual
    # population
    # t: max number of units in a subpopulation

    """Algorithm"""
    # Calculate the Euclidean distances from the individual to all others in the population
    distances = np.sqrt(np.sum((population - individual) ** 2, axis=1))
    # Get the indices of the individuals with the smallest distances
    closest_indices = np.argsort(distances)[:t]
    # Form the subpopulation with the closest individuals
    subpop = population[closest_indices]

    if show_distances == True:
        print(f'Distance: \n{distances[:t]}')
    if return_index == True:
        if t == 1:
            return closest_indices,subpop.flatten()
        else:
            return closest_indices,subpop
    else:
        if t == 1:
            return subpop.flatten()
        else:
            return subpop

# # Test the function
# # Assuming P is a numpy array of individuals where each individual is a point in n-dimensional space
# np.random.seed(0)
# P = np.random.rand(100, 2)  # Example: 100 individuals in a 2-dimensional space
# # print(f"P:{P}")
# print("")

# # The number of individuals to select with the smallest Euclidean distances
# t = 2
# # Forming subpopulations for each individual in P
# subpopulations = [subpopulating(xi, P, t) for xi in P]
# # print(f"subpopulations:{subpopulations}")
# print("")

# # Now subpopulations is a list of numpy arrays, each containing the t closest individuals to each xi in P (including xi itself)
# # For example, to access the subpopulation for the first individual in P:
# subpopulation_first_individual = subpopulations[0]
# print(f"subpopulation_first_individual:{subpopulation_first_individual}")

# [id1,id2],[v1,v2] = subpopulating(P[0],P,t,show_distances=True,return_index=True)
# v1,v2

In [10]:
def update_parameter(M_F,
                     M_CR,
                     Hm:int):
    """Input"""
    # MF: Historical memories of scaling factor of DE as F
    # MCR:Historical memories crossover rate of DE as CR
    # Hm: Size of Historical Memories

    # Randomly select an index
    hi = np.random.randint(0, Hm)
    # Generate Fi using the Cauchy distribution with the location parameter MF[hi] and scale 0.1
    Fi = np.random.standard_cauchy() * 0.1 + M_F[hi]
    # Generate CRi using the Gaussian distribution with mean MCR[hi] and standard deviation 0.1
    CRi = np.random.normal(M_CR[hi], 0.1)
    # Ensure CRi is within the range [0, 1] and Fi is within the range [0,2]
    Fi = np.clip(Fi, 0, 1)
    CRi = np.clip(CRi, 0, 1)
    return Fi, CRi

# # Coba
# MF = [0.5, 0.6, 0.7, 0.8, 0.9] 
# MCR = [0.1, 0.2, 0.3, 0.4, 0.5]
# update_parameter(MF,MCR,len(MF))

In [11]:
def meanWL(elements, weights):
    """
    Calculate the weighted Lehmer mean of elements.
    Lehmer mean is calculated as the weighted sum of the squares
    divided by the weighted sum of the elements.
    """
    numerator = np.sum(np.multiply(np.square(elements), weights))
    denominator = np.sum(np.multiply(elements, weights))
    return numerator / denominator if denominator != 0 else 0

# Define the weighted arithmetic mean function
def meanWA(elements, weights):
    """
    Calculate the weighted arithmetic mean of elements.
    This is the standard weighted mean.
    """
    return np.average(elements, weights=weights)

def update_history(M_F,M_CR,S_F,S_CR,k):
    weights = np.array([1 for _ in range (len(S_F))])
    if len(S_F)!=0:
        M_F[k] = meanWL(S_F,weights) 
    if len(S_CR)!=0:
        M_CR[k] = meanWA(S_CR,weights)
    return M_F,M_CR


In [12]:
np.random.seed(2)

# Main RADE algorithm loop
def SPO(archive, 
         objective_func,
         repulsion_func,
         bounds, 
         population_size, 
         max_generation, 
         num_l,
         epsilon,
         tau_d,
         s_max,
         print_gen = False):
    dimensions = len(bounds)
    population = generate_points(dim=dimensions,
                                 npoint=population_size,
                                 low=bounds[:,0],
                                 high=bounds[:,1],
                                 sobol=True)
    fitness = np.asarray([objective_func(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    subpopA = np.array([subpopulating(xi, population, num_l) for xi in population])
    # print(subpopA)
    for gen in range(max_generation):
        for i in range(population_size):
            x_i = population[i]
            trial_vector, trial_value = SpiralOpt(objective_function=root_objective_function,
                                                  set_of_points=subpopA[i],
                                                  r = 0.95,
                                                  theta=np.pi/4,
                                                  iter_max=50,
                                                  error_max=1e-5,
                                                  seed=19)
            trial_vector = np.array(trial_vector)
            trial_fitness = repulsion_func(trial_vector)
            
            if trial_fitness < fitness[i]:
                fitness[i] = trial_fitness
                population[i] = trial_vector
                archive = update_archive(x = trial_vector,
                                        objective_function=objective_func,
                                        archive=archive,
                                        epsilon=epsilon,
                                        tau_d=tau_d,
                                        s_max=s_max)
                if trial_fitness < fitness[best_idx]:
                    best_idx = i
                    best = trial_vector

        if print_gen == True:
            print(f"=========Generation {gen}=========")
            print(f"Best Fitness: {fitness[best_idx]}")
            print(f'Archive:{archive}')
    return best, fitness[best_idx], archiveA



archiveA = []
best_solution, best_fitness, archiveA = SPO(objective_func=root_objective_function, 
                                            repulsion_func=partial(repulsion_function, 
                                                                    archive=archiveA,
                                                                    objective_func=root_objective_function),
                                            archive=archiveA,
                                            bounds = boundaries, 
                                            population_size=250, 
                                            max_generation=2, 
                                            num_l=40,
                                            epsilon=1e-3,
                                            tau_d=0.4,
                                            s_max=100,
                                            print_gen=True)
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print(f'Roots: {archiveA}')

C:\Users\ACER\AppData\Local\Temp\ipykernel_19336\1671718438.py:41: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  diff = np.abs(np.sqrt(np.sum(num**2 for num in set_of_points[iter][i]))-np.sqrt(np.sum(num**2 for num in set_of_points[iter-1][i])))


=========Generation 0=========
Best Fitness: 3.1091049296567524e-11
Archive:[array([0.66711775, 0.69010405]), array([-0.15529293,  6.43984053]), array([-6.11708803, -0.16347596]), array([0.16333242, 6.12243497]), array([-6.44254222,  0.1552615 ]), array([-0.93210953,  1.06789057])]
=========Generation 1=========
Best Fitness: 3.1091049296567524e-11
Archive:[array([0.66711775, 0.69010405]), array([-0.15529293,  6.43984053]), array([-6.11708803, -0.16347596]), array([0.16333242, 6.12243497]), array([-6.44254222,  0.1552615 ]), array([-0.93210953,  1.06789057])]
Best Solution: [0.66711775 0.69010405]
Best Fitness: 3.1091049296567524e-11
Roots: [array([0.66711775, 0.69010405]), array([-0.15529293,  6.43984053]), array([-6.11708803, -0.16347596]), array([0.16333242, 6.12243497]), array([-6.44254222,  0.1552615 ]), array([-0.93210953,  1.06789057])]
